<a href="https://colab.research.google.com/github/Anna172/Mainor/blob/master/DomohozBaby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [30]:
data = pd.read_csv("DomStart.csv")
data.head()

,ID_W,ID_H,STATUS,A3,A3_3,NFM,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,B4_9_1,B4_9_2,B4_9_3,...,B19_9_9,B19_910,B19_911,B19_912,B19_913,B19_914,B19_915,B19_916,B19_917,B19_918,H45,H46_1B,H46_2B,H46_3B,H46_4B,H461_1,H461_2,H461_3,H461_4,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,YEAR
0,5,1001,3,1,NaN,2,1.0,1.0,NaN,NaN,2.0,1973.0,2.0,2.0,2.0,NaN,NaN,1.0,1971.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
1,5,1002,3,2,NaN,4,1.0,3.0,NaN,NaN,2.0,1955.0,3.0,2.0,4.0,NaN,NaN,1.0,1949.0,3.0,1.0,3.0,5.0,NaN,NaN,2.0,1984.0,NaN,4.0,4.0,4.0,6.0,NaN,NaN,1.0,1986.0,NaN,4.0,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
2,5,1003,3,3,NaN,1,1.0,7.0,NaN,NaN,2.0,1944.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
3,5,1004,3,4,NaN,1,1.0,8.0,NaN,NaN,2.0,1968.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994
4,5,1005,3,5,NaN,2,1.0,9.0,NaN,NaN,2.0,1946.0,2.0,2.0,10.0,NaN,NaN,1.0,1951.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994


In [31]:
data.shape


(131515, 331)

In [0]:
data = data.assign(Baby1=np.zeros(data.shape[0]))
data = data.assign(Baby2=np.zeros(data.shape[0]))
data = data.assign(Baby3=np.zeros(data.shape[0]))
data = data.assign(Baby4=np.zeros(data.shape[0]))
#data = data.assign(Baby5=np.zeros(data.shape[0]))
data = data.assign(Childs_All=np.zeros(data.shape[0])) 
data = data.assign(Mother=np.zeros(data.shape[0]))
data = data.assign(Father=np.zeros(data.shape[0]))
data = data.assign(Childs=np.zeros(data.shape[0]))


In [0]:
def get_curr_year(value):
    if value >= 9:
        return 1991 + value
    elif value <= 7:
        return 1989 + value
    elif value == 8:
        return 1998

In [0]:
class Member:
    def __init__(self, index):
        self.index = index
        self.gender = None
        self.mother = -1
        self.father = -1
        self.childs = []

In [0]:
from operator import attrgetter
member_id = ['IDIND' + str(v) for v in range(1, 25)]

In [0]:
for main_index in range(len(data)):
# for main_index in range(2):
    
    number_members = int(data.loc[main_index, 'NFM']) # Количество членов семьи
    members_data = []
 
    # Парсим по IDIND@
    for i in range(number_members):
        member = data.loc[main_index, member_id[i]:member_id[i+1]]
        members_data.append(member)

    # Инициализируем классы членов семьи
    members = [Member(i + 1) for i in range(number_members)]
    for i in range(2, number_members + 1):
        for j in range(i-1, 0, -1):
            #Ячейка  B@.9.@, при разных варианциях имеет разные форматы
            if (i >= 11 and i <= 16) and (j >= 10):
                cell = "B" + str(i) + "9_" + str(j)
            elif (i >= 17 and i <= 24) and (j >= 10):
                cell = "B" + str(i) + "_9" + str(j)
            else:
                cell = "B" + str(i) + "_9_" + str(j)
            
            #Если член семьи является отцом/матерью
            if data.loc[main_index, cell] == 2:
                members[i-1].childs.append(j-1)
                gender = "B" + str(i) + "_4"
                members[i-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                if data.loc[main_index, gender] == 1:
                    members[j-1].father = i-1
                elif data.loc[main_index, gender] == 2:
                    members[j-1].mother = i-1  
                    
            #Если член семьи является сыном/дочерью       
            elif data.loc[main_index, cell] == 4:
                members[j-1].childs.append(i-1)
                gender = "B" + str(j) + "_4"
                members[j-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                members[i-1].father = j
                if data.loc[main_index, gender] == 1:
                    members[i-1].father = j-1
                elif data.loc[main_index, gender] == 2:
                    members[i-1].mother = j-1
       
#     for mem in members:
#         print(mem.index, mem.father, mem.mother, mem.childs)
        
    # Выбираем детей старше 19 лет
    childs = []
    all_childs = []
    for i in range(number_members):
        year = "B" + str(i + 1) + "_5"
        if not members[i].childs and (members[i].father != -1 or members[i].mother != -1)\
        and (data.loc[main_index, 'YEAR'] - data.loc[main_index, year] < 26):
           # year = "B" + str(i + 1) + "_5"
            all_childs.append(members[i])
            if data.loc[main_index, year] != None and data.loc[main_index, 'ID_W'] != None \
            and (data.loc[main_index, 'YEAR'] - data.loc[main_index, year] > 19):
                childs.append(members[i])
    
    if not childs:
        continue
    if not all_childs:
        continue
    
    # Если получилось так, что есть несколько групп родителей-детей, то выбираем более многочисленную
    sorted_childs = sorted(childs, key=attrgetter('mother', 'father'))
    mother, father = sorted_childs[0].mother, sorted_childs[0].father
    sorted_all_childs = sorted(all_childs, key=attrgetter('mother', 'father'))
    
    last_group, curr_group = 0, 1
    index = 0
    
    for i in range(1, len(sorted_childs)):
        if sorted_childs[i].mother != mother or sorted_childs[i].father != father:
            if curr_group > last_group:   
                last_group = curr_group
                curr_group = 1
                index = i - 1
        else:
            curr_group += 1
    
    if curr_group > last_group:   
        last_group = curr_group
        curr_group = 1
        index = len(sorted_childs) - 1
    
    # last_group - самая большая группа родителей-детей
    # Инициализируем новые столбцы
    
    mother_index = sorted_childs[index].mother
    father_index = sorted_childs[index].father
    
    all_childs_group = 0

    for k in range(len(sorted_all_childs)):
        if sorted_all_childs[k].mother == mother_index or sorted_all_childs[k].father == father_index:
            all_childs_group += 1

            
    i = 1
    for k in childs:
        cell = "Baby" + str(i)
        Baby_ID = "IDIND" + str(k.index)
        
        if k.mother == mother_index and k.father == father_index:
            data.loc[main_index, cell] = data.loc[main_index, Baby_ID]
            i += 1

#     print(mother_index, father_index) Baby1 
    
    if mother_index != -1:
        mother_ID = "IDIND" + str(mother_index + 1)
        data.loc[main_index, 'Mother'] = data.loc[main_index, mother_ID]
    if father_index != -1: 
        father_ID = "IDIND" + str(father_index + 1)
        data.loc[main_index, 'Father'] = data.loc[main_index, father_ID]
    data.loc[main_index, 'Childs'] = last_group
  
    
    data.loc[main_index, 'Childs_All'] = all_childs_group
    

In [37]:
data.head()

,ID_W,ID_H,STATUS,A3,A3_3,NFM,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,B4_9_1,B4_9_2,B4_9_3,...,B19_917,B19_918,H45,H46_1B,H46_2B,H46_3B,H46_4B,H461_1,H461_2,H461_3,H461_4,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,YEAR,Baby1,Baby2,Baby3,Baby4,Childs_All,Mother,Father,Childs
0,5,1001,3,1,NaN,2,1.0,1.0,NaN,NaN,2.0,1973.0,2.0,2.0,2.0,NaN,NaN,1.0,1971.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,1002,3,2,NaN,4,1.0,3.0,NaN,NaN,2.0,1955.0,3.0,2.0,4.0,NaN,NaN,1.0,1949.0,3.0,1.0,3.0,5.0,NaN,NaN,2.0,1984.0,NaN,4.0,4.0,4.0,6.0,NaN,NaN,1.0,1986.0,NaN,4.0,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,1003,3,3,NaN,1,1.0,7.0,NaN,NaN,2.0,1944.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,1004,3,4,NaN,1,1.0,8.0,NaN,NaN,2.0,1968.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1005,3,5,NaN,2,1.0,9.0,NaN,NaN,2.0,1946.0,2.0,2.0,10.0,NaN,NaN,1.0,1951.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
data.shape


(131515, 339)

In [0]:
data.to_csv("DomData2.csv", index=False)

In [39]:
data[data['Baby4'] > 0]

,ID_W,ID_H,STATUS,A3,A3_3,NFM,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,B4_9_1,B4_9_2,B4_9_3,...,B19_917,B19_918,H45,H46_1B,H46_2B,H46_3B,H46_4B,H461_1,H461_2,H461_3,H461_4,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,YEAR,Baby1,Baby2,Baby3,Baby4,Childs_All,Mother,Father,Childs
77611,20,1070179,4,179,1.0,6,1.0,37836.0,1.0,NaN,2.0,1959.0,NaN,2.0,37837.0,1.0,NaN,1.0,1960.0,NaN,1.0,3.0,37838.0,1.0,NaN,1.0,1986.0,NaN,4.0,4.0,4.0,37839.0,1.0,NaN,1.0,1987.0,NaN,4.0,4.0,6.0,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2011,37838.0,37839.0,37840.0,37841.0,4.0,37836.0,37837.0,4.0
100205,23,770171,4,171,1.0,6,1.0,36276.0,1.0,NaN,2.0,1957.0,NaN,2.0,36277.0,1.0,NaN,1.0,1990.0,NaN,4.0,3.0,36278.0,1.0,NaN,2.0,1991.0,NaN,4.0,6.0,4.0,36279.0,1.0,NaN,2.0,1993.0,NaN,4.0,6.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014,36277.0,36278.0,36279.0,36280.0,5.0,36276.0,36277.0,4.0
106999,24,770171,4,171,1.0,6,1.0,36276.0,1.0,NaN,2.0,1957.0,NaN,2.0,36277.0,1.0,NaN,1.0,1990.0,NaN,4.0,3.0,36278.0,1.0,NaN,2.0,1991.0,NaN,4.0,6.0,4.0,36279.0,1.0,NaN,2.0,1993.0,NaN,4.0,6.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,36277.0,36278.0,36279.0,36280.0,5.0,36276.0,36277.0,4.0
